# Imports

In [ ]:
PROJECT_PATH = '.'

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from lib_tds_rg.tds_rg_module import StatisticalTests as stest
from lib_tds_rg.tds_rg_module import TheoreticalDistributionFitter as dist_fitter
from lib_tds_rg.tds_rg_module import DistributionPlotter as plotter
from lib_tds_rg.tds_rg_module import RGSimilarity as rg_sim
from lib_tds_rg.dataset_handling_utils import *
from lib_tds_rg.empirical_experiments_utils import *

mpl.style.use("ggplot")
warnings.filterwarnings("ignore")

# Constants

In [ ]:
SEED = 2022

# Handle datasets

<u>Source:</u>

*   Dataset 1 - [Houses price prediction](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)
*   Dataset 2 - [Cancer detection](https://www.kaggle.com/uciml/breast-cancer-wisconsin-data)
*   Dataset 3 - [Stress detection in sleep](https://www.kaggle.com/laavanya/human-stress-detection-in-and-through-sleep)
*   Dataset 4 - [Salaries in san fransisco ](https://www.kaggle.com/kaggle/sf-salaries)

## Load the datasets

In [ ]:
datasets_info = [
    (f"{DS.HOUSES}.csv", preprocess_houses),
    (f"{DS.CANCER}.csv", preprocess_cancer),
    (f"{DS.SLEEPING}.csv", preprocess_sleeping),
    (f"{DS.SALARIES}.csv", preprocess_salaries)
]

dtfs = {}
for dataset in datasets_info:
    full_file_name = dataset[0]
    file_name = full_file_name.split('.')[0]
    file_path = f"{PROJECT_PATH}/datasets/{full_file_name}"
    preprocess_func = dataset[1]
    dtfs[file_name] = read_dataset(file_path, preprocess_func)

# The problem

One of the usages of statistical tests is to help the scientists conclude whether a data is drawn from a specific population or not, and which distributions are identical.<p>
Even though they are widely used, those statistical tests perform badly on several use cases, and it can be seen in the following examples.<p>


- When the samples generated from the same distribution but with **slightly** different parameters, the p-values are basically zero.

In [ ]:
sample_size = 50_000
np.random.seed(SEED)
sample1 = np.random.normal(0, 1, sample_size)
np.random.seed(SEED)
sample2 = np.random.normal(0, 1.05, sample_size)

# Perform statistical test
tests = ['KS test', 'CVM test']
pvals = [stest.ks_2_sample_test(sample1, sample2), 
        stest.cvm_2_sample_test(sample1, sample2)]
tabulate_data(cols=[tests, pvals],
              headers=['Statistical Test', 'Value'],
              title='Results')
# Plot samples
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plotter.plot_ecdfs([sample1, sample2], ['sample1', 'sample2'])
plt.subplot(1, 2, 2)
plotter.plot_histograms_kdes([sample1, sample2], ['sample1', 'sample2'])

- When the data generated relatively similar distributions but not the same generator, p-values also small enough to be considered 0.

In [ ]:
np.random.seed(SEED)
sample_size = 50_000
sample1 = np.random.normal(1, 0.45, sample_size)
np.random.seed(SEED)
sample2 = np.random.lognormal(0, 0.5, sample_size)

# Perform statistical test
tests = ['KS test', 'CVM test']
pvals = [stest.ks_2_sample_test(sample1, sample2), 
        stest.cvm_2_sample_test(sample1, sample2)]
tabulate_data(cols=[tests, pvals],
              headers=['Statistical Test', 'Value'],
              title='Results')
# Plot samples
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plotter.plot_ecdfs([sample1, sample2], ['sample1', 'sample2'])
plt.subplot(1, 2, 2)
plotter.plot_histograms_kdes([sample1, sample2], ['sample1', 'sample2'])

- There are stability issues depending on the different statistical test, sample size, random seed, etc. <p>
Several runs with the same generator can **yield** very different p-values,<p> sometimes even lower than the minimum level needed to accept the null hypothesis.

In [ ]:
num_of_samples = 15
sample_size = 40_000
np.random.seed(SEED)
ks_pvalues = list()
cvm_pvalues = list()
for _ in range(num_of_samples):
    sample1 = np.random.normal(0, 1, sample_size)
    sample2 = np.random.normal(0, 1, sample_size)
    ks_pvalue = stest.ks_2_sample_test(sample1, sample2) 
    cvm_pvalue = stest.cvm_2_sample_test(sample1, sample2)
    ks_pvalues.append(ks_pvalue)
    cvm_pvalues.append(cvm_pvalue)

titles = ['KS - Result Statistics', 'CVM - Result Statistics']
values = [[np.mean(ks_pvalues), np.std(ks_pvalues), np.min(ks_pvalues), np.max(ks_pvalues)],
          [np.mean(cvm_pvalues), np.std(cvm_pvalues), np.min(cvm_pvalues), np.max(cvm_pvalues)]]
stats = [["ks_pvalues_mean", "ks_pvalues_std", "ks_pvalues_min", "ks_pvalues_max"], 
         ["cvm_pvalues_mean", "cvm_pvalues_std", "cvm_pvalues_min", "cvm_pvalues_max"]]

for i,v in enumerate(values):
    tabulate_data(cols=[stats[i], v], headers=['Statistics Metric', 'Value'], title=titles[i])

xs = list(range(1, num_of_samples + 1))
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
sns.barplot(xs, ks_pvalues)
plt.title('KS P-values')
plt.subplot(1, 2, 2)
sns.barplot(xs, cvm_pvalues)
plt.title('CVM P-values')

# Our solution


The basic idea of the method is to use KDEs of both distributions, to compute the normalized Root Mean Squared Error (RMSE) between the KDEs.
<br>
Our method supports scaling the data to [0,1], when the similarity needed between 2 distributions with different value ranges, e.g. height vs weight.

<u>Algorithm steps:</u>
```
    1. Scale the samples so they are on the same domain (optional)
    2. Calculate the KDE for each sample
    3. Evaluate the values of the KDEs on each of the data sample points
    4. Compute a root mean squared error between the KDEs values
    5. Normalize the result to range [0,1]
```

# Experiments

 ## Top K theoretical distributions view

 This comes to serve as a view of what are the top K theoretical
 distribution the data could be sampled from.<br>
 The theoretical CDF built by fitting the parameters using maximum likelihood estimation (MLE).<br>
 The scoring metric based on combined P-Values of Kolmogorov-Smirnov and Cramér-von Mises statistical tests using the Fisher method.

 <br>
 <u>Note:</u> This is not the main part of the project, but comes as a complementary view.

 <br>
 TODO: Think if we need to explain about CVM and Fisher method here.

In [ ]:
k = 3

### Dataset 1 - [Houses price prediction](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

In [ ]:
dtfs[DS.HOUSES].describe()

#### SalePrice - The property's sale price in dollars

In [ ]:
data = dtfs[DS.HOUSES].SalePrice
top_k_theoretical_dists = dist_fitter.top_k_theoretical_distributions(data, k, show_results=True)
plotter.plot_top_k(data, top_k_theoretical_dists)

### Dataset 2 - [Cancer detection](https://www.kaggle.com/uciml/breast-cancer-wisconsin-data)

In [ ]:
dtfs[DS.CANCER].describe()

#### Radius - Mean of distances from center to points on the perimeter

In [ ]:
data = dtfs[DS.CANCER].radius_mean
top_k_theoretical_dists = dist_fitter.top_k_theoretical_distributions(data, k, show_results=True)
plotter.plot_top_k(data, top_k_theoretical_dists)

### Dataset 3 - [Stress detection in sleep](https://www.kaggle.com/laavanya/human-stress-detection-in-and-through-sleep)

In [ ]:
dtfs[DS.SLEEPING].describe()

#### t - Body temperature

In [ ]:
data = dtfs[DS.SLEEPING].t
top_k_theoretical_dists = dist_fitter.top_k_theoretical_distributions(data, k, show_results=True)
plotter.plot_top_k(data, top_k_theoretical_dists)

### Dataset 4 - [Salaries in san fransisco ](https://www.kaggle.com/kaggle/sf-salaries)

In [ ]:
dtfs[DS.SALARIES].describe()

#### BasePay - The base pay of the salary

In [ ]:
data = dtfs[DS.SALARIES].BasePay.sample(2000)
top_k_theoretical_dists = dist_fitter.top_k_theoretical_distributions(data, k, show_results=True)
plotter.plot_top_k(data, top_k_theoretical_dists)

## Empirical distributions similarity

This part demonstrates our method to compute the similarity between 2 empirical distributions, as described in *'Our solution'* section above.

### Set experiments random seed
Note that there are additional seeds done some of the experiments.<br>
This is done in order to isolate the change of the experiment, so the test results won't be affected by the random state.

In [ ]:
np.random.seed(SEED)

### Synthetic data experiments

This section of the experiments shows how synthetic data samples compared with our method vs Kolmogorov-Smirnov and Cramér-von Mises statistical tests.

In [ ]:
sample_size = 1000

#### Experiment 1 - Normal distribution similarity

This experiment shows the comparison of 2 samples from the exact same generator(Normal distribution) with the same parameters. 

In [ ]:
dist_sample1 = np.random.normal(0, 1, sample_size)
dist_sample2 = np.random.normal(0, 1, sample_size)
show_2_sample_comparisons(dist_sample1, dist_sample2, samples_labels=['norm_s1','norm_s2'], show_ecdfs=True)

#### Experiment 2 - Normal distribution similarity with data scaling

This experiment shows the comparison of 2 samples from the same distribution function, but different parameters, i.e. different values domain.

In [ ]:
dist_sample1 = np.random.normal(170, 10, sample_size)  # e.g. heights in CM
dist_sample2 = np.random.normal(80, 5, sample_size)  # e.g. weights in KG
show_2_sample_comparisons(dist_sample1, dist_sample2, samples_labels=['norm_s1','norm_s2'], scale_data=True, show_ecdfs=True)

#### Experiment 3 - Chisquare distribution similarity

This experiment shows the comparison of 2 samples from the same distribution function(chisquare distribution), but different parameters with a small change.

In [ ]:
np.random.seed(SEED)
dist_sample1 = np.random.chisquare(3.7, sample_size)
np.random.seed(SEED)
dist_sample2 = np.random.chisquare(1, sample_size)
show_2_sample_comparisons(dist_sample1, dist_sample2, samples_labels=['chisquare_s1','chisquare_s2'], show_ecdfs=True)

#### Experiment 4 - Gamma distribution similarity

This experiment shows the comparison of 2 samples from the exact same generator(Gamma distribution) from right skewed distribution with the same parameters.

In [ ]:
dist_sample1 = np.random.gamma(2, 1, sample_size)
dist_sample2 = np.random.gamma(2, 1, sample_size)
show_2_sample_comparisons(dist_sample1, dist_sample2, samples_labels=['gamma_s1','gamma_s2'], show_ecdfs=True)

#### Experiment 5 - Geometric vs Beta distributions similarity

This experiment shows the comparison of 2 samples from different generators(Geometric and Beta distribution).

In [ ]:
np.random.seed(SEED)
dist_sample1 = np.random.geometric(0.5, sample_size)
np.random.seed(SEED)
dist_sample2 = np.random.beta(3, 20, sample_size)
show_2_sample_comparisons(dist_sample1, dist_sample2, samples_labels=['geometric','beta'], show_ecdfs=True)

#### Experiment 6 - Multiple samples from same generator

This experiment shows the comparison of 2 samples from the exact same generator(Normal distribution) sampled multiple times.

In [ ]:
def run_multiple_samples_experiment(sample_size, num_of_samples):
    np.random.seed(SEED)
    rg_scores = list()
    ks_pvalues = list()
    cvm_pvalues = list()
    for _ in range(num_of_samples):
        dist_sample1 = np.random.normal(0, 1, sample_size)
        dist_sample2 = np.random.normal(0, 1, sample_size)
        rg_score, ks_pvalue, cvm_pvalue = get_empirical_comparison(dist_sample1, dist_sample2)
        rg_percent_similarity = (1 - rg_score) * 100
        rg_scores.append(rg_percent_similarity)
        ks_pvalues.append(ks_pvalue)
        cvm_pvalues.append(cvm_pvalue)

    log_multi_test_statistics(rg_scores, ks_pvalues, cvm_pvalues, scores_units=['[%]']*4)

##### Small sample size

In [ ]:
sample_size = 50
num_of_samples = 10
run_multiple_samples_experiment(sample_size, num_of_samples)

##### Large sample size

In [ ]:
sample_size = 1000
num_of_samples = 10
run_multiple_samples_experiment(sample_size, num_of_samples)

#### Experiment 7 - Same generator, gradual change

This experiment shows the comparison of 2 samples from the same distribution generator(Normal distribution) function with the parameters gradually changing.

In [ ]:
sample_size = 500
num_of_samples = 10
rg_scores = list()
ks_pvalues = list()
cvm_pvalues = list()
mean = 0
std = 1
mean_step = 1
std_step = 1
np.random.seed(SEED)
dist_sample1 = np.random.normal(mean, std, sample_size)
for _ in range(num_of_samples):
    np.random.seed(SEED)
    mean += mean_step
    std += std_step
    dist_sample2 = np.random.normal(mean, std, sample_size)
    rg_score, ks_pvalue, cvm_pvalue = get_empirical_comparison(dist_sample1, dist_sample2)
    rg_scores.append(-1/np.log(1-rg_score))
    ks_pvalues.append(-1/np.log(ks_pvalue))
    cvm_pvalues.append(-1/np.log(cvm_pvalue))

plot_titles = ["-1/log(KS-Pvalues)", "-1/log(CVM-Pvalues)", "-1/log(1 - RG-Similarity Scores)"]
log_multi_test_statistics(rg_scores, ks_pvalues, cvm_pvalues, plot_titles=plot_titles)

## Real world data experiments
This section of the experiments shows how real world datasets compared with our method vs Kolmogorov-Smirnov and Cramér-von Mises statistical tests.
<br>
See explanation about the experiments in the experiments functions below.

### Experiments functions

#### Single sample experiment

This function demonstrates an experiment of splitting a datasett to two groups, e.g. (train 80[%], test 20[%]), as an example of usage.

In [ ]:
def run_single_sample_real_dataset_experiment(data, sample_size, scale_data=False, show_ecdfs=False):
    dist_sample = data.sample(sample_size).values
    dist_sample1 = dist_sample[0:int(0.2*sample_size)]
    dist_sample2 = dist_sample[int(0.2*sample_size):]
    show_2_sample_comparisons(dist_sample1, dist_sample2, scale_data=scale_data, show_ecdfs=show_ecdfs)

#### Two sample experiment

This function demonstrates an experiment of comparing 2 different features distributions as an example of usage.

In [ ]:
def run_two_sample_real_dataset_experiment(data_1, data_2, scale_data=False, show_ecdfs=False):
    show_2_sample_comparisons(data_1.to_numpy(), data_2.to_numpy(), scale_data=scale_data, show_ecdfs=show_ecdfs)

#### Multiple samples experiment

This function demonstrates an experiment of splitting a dataset to two groups, e.g. (train80[%], test 20[%]), as an example of usage.
<br>
It runs the experiment multiple times, in order to see the results stability between different samples. As an example of usage, could be used for k-Fold Cross Validation.

In [ ]:
def run_multiple_samples_real_dataset_experiment(data, sample_size, num_of_samples, scale_data=False):
    np.random.seed(SEED)
    rg_scores = list()
    ks_pvalues = list()
    cvm_pvalues = list()
    for _ in range(num_of_samples):
        dist_sample = data.sample(sample_size).values
        dist_sample1 = dist_sample[0:int(0.2*sample_size)]
        dist_sample2 = dist_sample[int(0.2*sample_size):]
        rg_score, ks_pvalue, cvm_pvalue = get_empirical_comparison(dist_sample1, dist_sample2, scale_data=scale_data)
        rg_percent_similarity = (1 - rg_score) * 100
        rg_scores.append(rg_percent_similarity)
        ks_pvalues.append(ks_pvalue)
        cvm_pvalues.append(cvm_pvalue)

    log_multi_test_statistics(rg_scores, ks_pvalues, cvm_pvalues, scores_units=['[%]']*4)

### Dataset 1 - [Houses price prediction](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

In [ ]:
dtfs[DS.HOUSES].describe()

In [ ]:
sample_size = 1400
num_of_samples = 10

#### Experiment 1: Single sample - SalePrice - The property's sale price in dollars


In [ ]:
data = dtfs[DS.HOUSES].SalePrice
run_single_sample_real_dataset_experiment(data, sample_size, show_ecdfs=True)

#### Experiment 2: Multiple samples - SalePrice - The property's sale price in dollars


In [ ]:
data = dtfs[DS.HOUSES].SalePrice
run_multiple_samples_real_dataset_experiment(data, sample_size, num_of_samples)

#### Experiment 3: Single sample - LotArea - Lot size in square feet

In [ ]:
data = dtfs[DS.HOUSES].LotArea
run_single_sample_real_dataset_experiment(data, sample_size, show_ecdfs=True)

#### Experiment 4: Multiple samples - LotArea - Lot size in square feet

In [ ]:
data = dtfs[DS.HOUSES].LotArea
run_multiple_samples_real_dataset_experiment(data, sample_size, num_of_samples)

#### Experiment 5: Single sample - SalePrice vs LotArea comparison

This experiment shows the distribution similarity between 2 different features of the dataset with and without scaling.

In [ ]:
data_1 = dtfs[DS.HOUSES].SalePrice
data_2 = dtfs[DS.HOUSES].LotArea
run_two_sample_real_dataset_experiment(data_1, data_2, scale_data=True, show_ecdfs=True)

### Dataset 2 - [Cancer detection](https://www.kaggle.com/uciml/breast-cancer-wisconsin-data)

In [ ]:
dtfs[DS.CANCER].describe()

In [ ]:
sample_size = 569
num_of_samples = 10

#### Experiment 1: Single sample - Radius - Mean of distances from center to points on the perimeter


In [ ]:
data = dtfs[DS.CANCER].radius_mean
run_single_sample_real_dataset_experiment(data, sample_size, show_ecdfs=True)

#### Experiment 2: Multiple samples - Radius - Mean of distances from center to points on the perimeter


In [ ]:
data = dtfs[DS.CANCER].radius_mean
run_multiple_samples_real_dataset_experiment(data, sample_size, num_of_samples)

#### Experiment 3: Single sample - Concavity - Severity of concave portions of the contour

In [ ]:
data = dtfs[DS.CANCER].concavity_mean
run_single_sample_real_dataset_experiment(data, sample_size, show_ecdfs=True)

#### Experiment 4: Multiple samples - Concavity - Severity of concave portions of the contour

In [ ]:
data = dtfs[DS.CANCER].concavity_mean
run_multiple_samples_real_dataset_experiment(data, sample_size, num_of_samples)

#### Experiment 5: Single sample - Radius vs Concavity comparison

This experiment shows the distribution similarity between 2 different features of the dataset with and without scaling.

In [ ]:
data_1 = dtfs[DS.CANCER].radius_mean
data_2 = dtfs[DS.CANCER].concavity_mean
run_two_sample_real_dataset_experiment(data_1, data_2, scale_data=True, show_ecdfs=True)

### Dataset 3 - [Stress detection in sleep](https://www.kaggle.com/laavanya/human-stress-detection-in-and-through-sleep)

In [ ]:
dtfs[DS.SLEEPING].describe()

In [ ]:
sample_size = 630
num_of_samples = 10

#### Experiment 1: Single sample - sr - Snoring range of the user


In [ ]:
data = dtfs[DS.SLEEPING].sr
run_single_sample_real_dataset_experiment(data, sample_size, show_ecdfs=True)

#### Experiment 2: Multiple samples - sr - Snoring range of the user


In [ ]:
data = dtfs[DS.SLEEPING].sr
run_multiple_samples_real_dataset_experiment(data, sample_size, num_of_samples)

#### Experiment 3: Single sample - t - Body temperature

In [ ]:
data = dtfs[DS.SLEEPING].t
run_single_sample_real_dataset_experiment(data, sample_size, show_ecdfs=True)

#### Experiment 4: Multiple samples - t - Body temperature

In [ ]:
data = dtfs[DS.SLEEPING].t
run_multiple_samples_real_dataset_experiment(data, sample_size, num_of_samples)

#### Experiment 5: Single sample - sr vs t comparison

This experiment shows the distribution similarity between 2 different features of the dataset with and without scaling.

In [ ]:
data_1 = dtfs[DS.SLEEPING].sr
data_2 = dtfs[DS.SLEEPING].t
run_two_sample_real_dataset_experiment(data_1, data_2, scale_data=True, show_ecdfs=True)

### Dataset 4 - [Salaries in san fransisco ](https://www.kaggle.com/kaggle/sf-salaries)

In [ ]:
dtfs[DS.SALARIES].describe()

In [ ]:
sample_size = 10000
num_of_samples = 10

#### Experiment 1: Single sample - BasePay - The base pay of the salary


In [ ]:
data = dtfs[DS.SALARIES].BasePay.sample(sample_size)
run_single_sample_real_dataset_experiment(data, sample_size, show_ecdfs=True)

#### Experiment 2: Multiple samples - BasePay - The base pay of the salary


In [ ]:
data = dtfs[DS.SALARIES].BasePay.sample(sample_size)
run_multiple_samples_real_dataset_experiment(data, sample_size, num_of_samples)

#### Experiment 3: Single sample - OvertimePay - The pay for overtime hours

In [ ]:
data = dtfs[DS.SALARIES].OvertimePay.sample(sample_size)
run_single_sample_real_dataset_experiment(data, sample_size, show_ecdfs=True)

#### Experiment 4: Multiple samples - OvertimePay - The pay for overtime hours

In [ ]:
data = dtfs[DS.SALARIES].OvertimePay.sample(sample_size)
run_multiple_samples_real_dataset_experiment(data, sample_size, num_of_samples)

#### Experiment 5: Single sample - BasePay vs OvertimePay comparison

This experiment shows the distribution similarity between 2 different features of the dataset with and without scaling.

In [ ]:
data_1 = dtfs[DS.SALARIES].BasePay.sample(sample_size)
data_2 = dtfs[DS.SALARIES].OvertimePay.sample(sample_size)
run_two_sample_real_dataset_experiment(data_1, data_2, scale_data=True, show_ecdfs=True)